# CNN

ILSVRC에서 우승을 했던 CNN 구조들은 AlexNet, VGG, GoogLeNet, ResNet이 있다.

초창기에 나온 것은 AlexNet이다. 

In [20]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim


class DbgLayer(nn.Module):
    def __init__(self):
        super(DbgLayer, self).__init__()
    
    def DbgPrint(self, result):
        print(result.size())

class AlexNet(nn.Module):
    def __init__(self, class_num):
        super(AlexNet, self).__init__()
        self.feature_network = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size = (11, 11), stride = 4, padding = 2),
            nn.ReLU(inplace = True),
#             DbgLayer(),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        self.classifier_network = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, class_num)
        )
        
    def forward(self, x):
        print('[] forward')
        x = self.feature_network(x)
        x = x.view(-1, 256 * 6 * 6)
        x = self.classifier_network(x)
        print(x.size())
            
        
def main():
    AN = AlexNet(10)
    param = list(AN.parameters())
    
    print('[] Network\n', AN)
    
    for p in param:
        print("[] Type: {},  size: {}".format(type(p), p.size()))

    ## training 
    input = torch.randn(1, 3, 227, 227)
    print(input.size())
    AN.forward(input)
#     print(AN.feature_network[0].weight.size())

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)


    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(AN.parameters(), lr=0.001, momentum=0.9)

#     print(trainloader)
    
#     t = iter(trainloader)
#     for i in range(0, 10):
#         images, labels = t.next()
#         print(labels)
#         print(images.size())
    for epoch in range(2):  # 데이터셋을 수차례 반복합니다.
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
        # 입력을 받은 후
            inputs, labels = data
            print('[] input size: ', inputs.size())
            
            inputs = input.view(4, 3, 64, 64)
            print('[] input size: ', inputs.size())

        # 변화도(Gradient) 매개변수를 0으로 만든 후
            optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
            outputs = AN.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # 통계 출력
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
            
    
if __name__ == '__main__':
    main()
        

[] Network
 AlexNet(
  (feature_network): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier_network): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in

RuntimeError: invalid argument 2: size '[4 x 3 x 64 x 64]' is invalid for input with 154587 elements at ..\aten\src\TH\THStorage.cpp:84